In [1]:
using Pkg
Pkg.activate("D:\\Science\\Code\\Julia\\envr")
using StaticArrays, DifferentialEquations, DynamicalSystems, JLD

  Activating project at `D:\Science\Code\Julia\envr`


In [2]:
function TM(u, p, t)

    U(y) = p[8] + p[9] / ( 1.0 + exp( -50.0 * (y - p[7]) ) )
    σ(x) = 1.0 / ( 1.0 + exp( -20.0 * (x-p[6]) ) )
    
    # u[1] - E; u[2] - x; u[3] - y
    # α - 1, τ - 2, τD - 3, τy - 4, J - 5, xthr - 6, ythr - 7, U0 - 8, ΔU0 - 9, β - 10, I0 -11

    du1 = (-u[1] + p[1] * log( 1.0 + exp( (p[5] * U(u[3]) * u[2] * u[1] + p[11]  ) / (p[1]) ) ) ) / p[2]
    du2 = (1.0 - u[2])/p[3] - U(u[3])*u[2]*u[1]
    du3 = (-u[3])/p[4] + p[10] * σ(u[2])
    return SA[du1, du2, du3]
end;

In [3]:
function output(idx_I0, idx_U0, I0, U0, u0)
    println("Индекс I0: $idx_I0; I0: $I0");flush(stdout)
    println("Индекс U0: $idx_U0; U0: $U0");flush(stdout)
    println("Начальные условия: $u0");flush(stdout)
end
function output(idx_U0,U0, u0)
    println("Индекс U0: $idx_U0; U0: $U0");flush(stdout)
    println("Начальные условия: $u0");flush(stdout)
end
function output_end_iter(ΛΛ, u0_lc)
    println("Спектр: $(ΛΛ)");flush(stdout)
    println("Последняя точка: $(u0_lc)");flush(stdout)
end
function separate()
    println();flush(stdout)
    println("---------");flush(stdout)
end;

In [4]:
t = 1000.0; tt = 250.0; tstep = 0.001
time_attract = 100.0
integ_set = (alg = RK4(), adaptive = false, dt = tstep);

In [5]:
const τ = 0.013;  const τD = 0.080;  const τy = 3.3;  const J = 3.07;  const β = 0.300
const xthr = 0.75; const ythr = 0.4
const α = 1.58;  U0 = 0.3;  const ΔU0 = 0.305; I0 = -1.40;

In [6]:
u0 = SA[8.39353, 0.73803, 0.429055]

3-element SVector{3, Float64} with indices SOneTo(3):
 8.39353
 0.73803
 0.429055

In [17]:
I0range = range( -1.40, -1.709, length = 400 )
U0range = range( 0.3,  0.47, length = 400)

0.3:0.0004260651629072682:0.47

In [18]:
Λs = zeros(length(I0range), length(U0range), 3)
u0s = zeros(length(I0range), length(U0range), 3);

In [19]:
const I0_start_value = I0range[1]

function init_ds(U0_, u0_lc)
    p = SA[α, τ, τD, τy, J, xthr, ythr, U0_, ΔU0, β, I0_start_value]
    ds = ContinuousDynamicalSystem(TM, u0_lc, p)
    return ds
end
function init_ds(U0_, I0_, u0_lc)
    p = SA[α, τ, τD, τy, J, xthr, ythr, U0_, ΔU0, β, I0_]
    ds = ContinuousDynamicalSystem(TM, u0_lc, p)
    return ds
end
function goto_attractor(ds_, time_attract)
    tr = trajectory(ds_, time_attract; Ttr = tt, diffeq = integ_set)
    u0_lc = tr[end]
    return u0_lc
end
function spectrum(ds_)
    ΛΛ = lyapunovspectrum(ds_, t; diffeq = integ_set)
    return ΛΛ
end
function save_output(idx_U0, ΛΛ, u0_lc)
    Λs[1, idx_U0, :] = ΛΛ
    u0s[1, idx_U0, :] = u0_lc
end
function save_output(index_I0, idx_U0, ΛΛ, u0_lc)
    Λs[index_I0, idx_U0, :] = ΛΛ
    u0s[index_I0, idx_U0, :] = u0_lc
end
function save_tofile()
    save("LSs_400x400.jld", "data", Λs)
    save("u0s_400x400.jld", "data", u0s)
end

save_tofile (generic function with 1 method)

In [20]:
for (idx_U0, U0_) in enumerate(U0range)
    
    if idx_U0 == 1 # Проверка на первую итерацию
        # Если использовать while условие не нужно!
        global u0_lc = u0
    end
    
    # output(idx_U0,U0_, u0_lc)
    
    ds = init_ds(U0_, u0_lc) # Инициализация системы
    u0_lc = goto_attractor(ds, time_attract) # Выход на аттрактор
    ds = init_ds(U0_, u0_lc)
    ΛΛ = spectrum(ds) # Вычисление спектра
    
    # output_end_iter(ΛΛ, u0_lc)
    
    save_output(idx_U0, ΛΛ, u0_lc) # Сохранение значений
    save_tofile()
    
    # separate()
end

In [21]:
for (idx_U0, U0_) in enumerate(U0range)
    for (idx_I0, I0_) in enumerate(I0range)
        
        if idx_I0 == 1 # Если использовать while условие не нужно!
            continue
        end
        
        
        u0_lc = u0s[idx_I0 - 1, idx_U0, :]
        
        # output(idx_I0, idx_U0, I0_, U0_, u0_lc)
        
        ds = init_ds(U0_, I0_, u0_lc)
        u0_lc = goto_attractor(ds, time_attract) # Выход на аттрактор
        ds = init_ds(U0_, I0_, u0_lc) 
        ΛΛ = spectrum(ds) # Вычисление спектра
        
        # output_end_iter(ΛΛ, u0_lc)
        
        save_output(idx_I0, idx_U0, ΛΛ, u0_lc)
        save_tofile()
        
        # separate()
    end
end

Error encountered while save FileIO.File{FileIO.DataFormat{:JLD}, String}("LSs_400x400.jld").

Fatal error:


LoadError: HDF5.API.H5Error: Error creating file LSs_400x400.jld
libhdf5 Stacktrace:
 [1] [0m[1mH5FD__sec2_open[22m: File accessibility/Unable to open file
[90m     unable to open file: name = 'LSs_400x400.jld', errno = 22, error message = 'Invalid argument', flags = 13, o_flags = 302[39m
  ⋮
Stacktrace:
  [1] [0m[1mmacro expansion[22m
[90m    @ [39m[90mC:\Users\HOME\.julia\packages\HDF5\HtnQZ\src\api\[39m[90m[4merror.jl:18[24m[39m[90m [inlined][39m
  [2] [0m[1mh5f_create[22m[0m[1m([22m[90mpathname[39m::[0mString, [90mflags[39m::[0mUInt16, [90mfcpl_id[39m::[0mHDF5.FileCreateProperties, [90mfapl_id[39m::[0mHDF5.FileAccessProperties[0m[1m)[22m
[90m    @ [39m[35mHDF5.API[39m [90mC:\Users\HOME\.julia\packages\HDF5\HtnQZ\src\api\[39m[90m[4mfunctions.jl:1070[24m[39m
  [3] [0m[1mjldopen[22m[0m[1m([22m[90mfilename[39m::[0mString, [90mrd[39m::[0mBool, [90mwr[39m::[0mBool, [90mcr[39m::[0mBool, [90mtr[39m::[0mBool, [90mff[39m